In [1]:
class HomeworkError extends Error {
    
    static ERROR_CODE = {
        NOT_A_PRODUCT: 'NOT A PRODUCT',
        NOT_IN_CART: 'NOT IN CART',
        EXPIRED: 'EXPIRED',
        POOR_ALERT: 'NO MONEY'
    };

    static ERROR_MESSAGE = {
        [HomeworkError.ERROR_CODE.NOT_A_PRODUCT]: 'The method must receive an instance of product!',
        [HomeworkError.ERROR_CODE.NOT_IN_CART]: 'The product must exist in the cart!',
        [HomeworkError.ERROR_CODE.EXPIRED]: 'The aliment is expired!',
        [HomeworkError.ERROR_CODE.POOR_ALERT]: 'The customer does not have enough money!',
    };
    
    constructor({ code, message = HomeworkError.ERROR_MESSAGE[code] }) {
        super(message);
        this.code = code;
    }
    
    toString() {
        return this.message + '\n' + this.stack;
    }
    
}

In [2]:
class Product {
    
    constructor({name, price}) {
        this.name = name;
        this.price = price;
    }
    
}


class Aliment extends Product {

    constructor({name, price, expirationDate, grammage}) {
        super({name, price});
        this.expirationDate = expirationDate;
        this.grammage = grammage;
    }
    
    isExpired() {
        // compare expiration date with current date
        return this.expirationDate < new Date();
    }

}


class Furniture extends Product {
 
    constructor({name, price, colour}) {
        super({name, price});
        this.colour = colour;
    }

}

In [3]:
class Person {
    
}


class Customer extends Person {
    
    constructor({money, shoppingCart}) {
        super({});
        this.money = money;
        this.shoppingCart = shoppingCart;
    }
    
    addToCart(product) {
        if (product instanceof Product) {
            if(product instanceof Aliment && product.isExpired()) {
                throw new HomeworkError({
                    code: HomeworkError.ERROR_CODE.EXPIRED
                });
            }
            // add the product to the cart
            this.shoppingCart.push(product);
        } else {
            throw new HomeworkError({
                code: HomeworkError.ERROR_CODE.NOT_A_PRODUCT
            });
        }
    }
    
    removeFromCart(product) {
        if (product instanceof Product) {
            // find element in shopping cart
            var idx = this.shoppingCart.findIndex((el) => el.toString() === product.toString())
            // remove it if it exists
            if(idx > -1) {
                this.shoppingCart.splice(idx, 1);
            } else {
                throw new HomeworkError({
                    code: HomeworkError.ERROR_CODE.NOT_IN_CART
                });
            }
        } else {
            throw new HomeworkError({
                code: HomeworkError.ERROR_CODE.NOT_A_PRODUCT
            });
        }
    }
    
}


class Cashier extends Person {
    
    constructor({totalSales}) {
        super({});
        this.totalSales = totalSales;
    }
    
    scanAndCash(customer) {
        let total = 0;
        // compute total amount to pay
        for (let prod of customer.shoppingCart) {
            total = total + prod.price;
        }
        // check if the customer affords the cart
        if(customer.money >= total) {
            customer.money = customer.money - total;
            console.log("purchase done");
            customer.shoppingCart.splice(0,customer.shoppingCart.length);
        } else {
            throw new HomeworkError({
                code: HomeworkError.ERROR_CODE.POOR_ALERT
            });
        }
    }
    
}

In [4]:
let customer1 = new Customer({money: 100, shoppingCart: new Array()});
let prod0 = new Furniture({name: "sofa", price: 70, colour: "blue"});
let prod1 = new Aliment({name: "pudding", price: 10, expirationDate: new Date() + 1, grammage: 50});
let prod2 = new Aliment({name: "puddingx3", price: 30, expirationDate: new Date() + 1, grammage: 150});
let prod3 = new Aliment({name: "puddingx3", price: 30, expirationDate: new Date() - 7, grammage: 150});


let cashier1 = new Cashier({totalSales: 0});

console.log('---------------------------------------');

// ok
try {
    customer1.addToCart(prod0);
    customer1.addToCart(prod1);
    cashier1.scanAndCash(customer1);
    console.log(customer1.money);
} catch (err) {
    console.log(err);
    customer1.shoppingCart.splice(0,customer1.shoppingCart.length);
}

console.log('---------------------------------------');

// error because of addToCart
try {
    customer1.addToCart(prod3);
    cashier1.scanAndCash(customer1);
    console.log(customer1.money);
} catch (err) {
    console.log(err);
    customer1.shoppingCart.splice(0,customer1.shoppingCart.length);
}

console.log('---------------------------------------');

// error because of scanAndCash
try {
    customer1.addToCart(prod0);
    customer1.addToCart(prod1);
    customer1.addToCart(prod2);
    cashier1.scanAndCash(customer1);
    console.log(customer1.money);
} catch (err) {
    console.log(err);
    customer1.shoppingCart.splice(0,customer1.shoppingCart.length);
}

console.log('---------------------------------------');

---------------------------------------
purchase done
20
---------------------------------------
HomeworkError: The aliment is expired!
    at Customer.addToCart (evalmachine.<anonymous>:17:23)
    at evalmachine.<anonymous>:27:15
    at Script.runInThisContext (node:vm:129:12)
    at Object.runInThisContext (node:vm:305:38)
    at run ([eval]:1020:15)
    at onRunRequest ([eval]:864:18)
    at onMessage ([eval]:828:13)
    at process.emit (node:events:527:28)
    at emit (node:internal/child_process:936:14)
    at process.processTicksAndRejections (node:internal/process/task_queues:83:21) {
  code: 'EXPIRED'
}
---------------------------------------
HomeworkError: The customer does not have enough money!
    at Cashier.scanAndCash (evalmachine.<anonymous>:71:19)
    at evalmachine.<anonymous>:42:14
    at Script.runInThisContext (node:vm:129:12)
    at Object.runInThisContext (node:vm:305:38)
    at run ([eval]:1020:15)
    at onRunRequest ([eval]:864:18)
    at onMessage ([eval]:828: